In [ ]:
import json
import matplotlib.path as mplPath
import numpy as np
import re
import matplotlib.pyplot as plt
from matplotlib.path import Path
import matplotlib.patches as patches
from dbbase import session,Dist,MDist,Cell,Record

json_data = open('beijing.json').read()
data = json.loads(json_data)

In [2]:
def hashlist(l):
    return str(l[0])+str(l[1])

In [3]:
cells=session.query(Cell).filter(Cell.avg_price>3000).all()
len(cells)

7147

In [4]:
cood=[]
for cell in cells:
    se=re.findall("[0-9.]+",cell.cood)
    if (se):
        cood.append([float(se[1]),float(se[0])])
    else:
        cood.append([0,0])

In [5]:
n=len(data['features'])

In [9]:
polys=[]
for i in range(0,n):
    poly=[]
    cdatas=data['features'][i]['geometry']['coordinates']
    if (data['features'][i]['properties']['childNum']<2):
        poly.append(mplPath.Path(np.array(cdatas[0])))
    else:
        for cdata in cdatas:
            poly.append(mplPath.Path(np.array(cdata[0])))
    polys.append(poly)

In [10]:
dists=[[] for i in range(0,n)]
dicts=[{} for i in range(0,n)]
for i in range(0,len(cells)):
    loc=cood[i]
    for j in reversed(range(0,n)):
        for poly in polys[j]:
            if (poly.contains_point(loc)):
                if(hashlist(loc) in dicts[j]):
                    dicts[j][hashlist(loc)]["name"].append(cells[i].name)
                    dicts[j][hashlist(loc)]["price"].append(cells[i].avg_price)
                else:
                    dicts[j][hashlist(loc)]={"cood":loc}
                    dicts[j][hashlist(loc)]["name"]=[cells[i].name]
                    dicts[j][hashlist(loc)]["price"]=[cells[i].avg_price]

In [12]:
for i in range(0,n):
    s=0
    num=0
    cell={'cood':[],}
    for key, value in dicts[i].iteritems():
        s+=sum(value["price"])
        num+=len(value["price"])
    data['features'][i]['properties']['avg_price']=s/num

24450
27549
26202
33667
33441
39476
40847
37162
21809
45938
65933
80854
55042
50434
111870
97030
88160
80216


In [ ]:
sum([1,2])

In [ ]:
data_array=np.array(data['features'][7]['geometry']['coordinates'][1])
path = Path(data_array)

fig = plt.figure()
ax = fig.add_subplot(111)
patch = patches.PathPatch(path, facecolor='orange', lw=2)
ax.add_patch(patch)
ax.set_xlim(max(data_array[:,0]),min(data_array[:,0]))
ax.set_ylim(max(data_array[:,1]),min(data_array[:,1]))
plt.show()

In [ ]:
max(data_array[:,0])